# Data Exploration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dateparse = lambda x: pd.datetime.strptime(x, '%Y%m%d')
with open('170101_190131過去電力供需資訊.csv', 'r') as f:
    data = pd.read_csv(f, parse_dates=['日期'], date_parser=dateparse, index_col=0)
data['Weekday'] = data.index.weekday
y_data = pd.Series(data['尖峰負載(MW)'],index=data.index)

plt.figure(figsize = (20,5))
plt.plot(y_data['2017'])
plt.title('2017 Peaking Power')
plt.figure(figsize = (20,5))
plt.plot(y_data['2018'])
plt.title('2018 Peaking Power')

  * 先觀察電力尖峰負載根據時間變化的趨勢，發現在夏季明顯升高

## 電力尖峰負載與氣溫

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y/%m/%d')
with open('Taipei_temperature.csv', 'r') as f:   
     weather = pd.read_csv(f, parse_dates=['date'], date_parser=dateparse, index_col=0)
temper = pd.Series(weather['Temperature'],index=weather.index)
min_temper = pd.Series(weather['T Min'],index=weather.index)
max_temper = pd.Series(weather['T Max'],index=weather.index)

  * 讀入每日平均、最低、最高溫度的資料
  * 選用台北測站的資料，因為台北是民生用電佔比最多的地區

### 平均溫度和尖峰負載相關係數

In [ ]:
temper['2017'].corr(y_data['2017'])

In [ ]:
temper['2018'].corr(y_data['2018'])

### 最高溫度和尖峰負載相關係數

In [ ]:
max_temper['2017'].corr(y_data['2017'])

In [ ]:
max_temper['2018'].corr(y_data['2018'])

### 最低溫度和尖峰負載相關係數

In [ ]:
min_temper['2017'].corr(y_data['2017'])

In [ ]:
min_temper['2018'].corr(y_data['2018'])

  * 最低溫度相對於平均溫度和最高溫度來說，和電力尖峰負載相關性更高

In [ ]:
fig17 = plt.figure(figsize = (20,5))
ax1 = fig17.add_subplot(111)
ax1.plot(y_data['2017'],color='steelblue',label='Peaking Power')
ax1.legend(loc='upper right')
ax2 = ax1.twinx()
ax2.plot(min_temper['2017'],color='coral',label='Min Temperature')
ax2.legend(loc='upper left')
plt.title('2017 Peaking Power and Min Temperature')

fig18 = plt.figure(figsize = (20,5))
ax1 = fig18.add_subplot(111)
ax1.plot(y_data['2018'],color='steelblue',label='Peaking Power')
ax1.legend(loc='upper right')
ax2 = ax1.twinx()
ax2.plot(min_temper['2018'],color='coral',label='Min Temperature')
ax2.legend(loc='upper left')
plt.title('2018 Peaking Power and Min Temperature')

  * 由上圖可以推測每日最低溫度和電力尖峰負載正相關

## 電力尖峰負載與星期幾

In [ ]:
name = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
y_week = [None] * 7
plt.figure(figsize = (20,5))
y_data1718 = y_data['2017':'2018']
min_temper1718 = min_temper['2017':'2018']
for day in range(7):
    y_week[day] = y_data1718[data['Weekday'] == day]  
    plt.plot(y_week[day], label=name[day])
plt.legend(loc='upper right')   
plt.title('Peaking Power on Different Weekday')

  * 將一週 7 天的電力尖峰負載分開繪圖，發現週六、日的電力尖峰負載明顯比週間日低

In [ ]:
temp_week = [None] * 7

for day in range(7):
    y_week[day] = y_data1718[data['Weekday'] == day]  
    temp_week[day] = min_temper1718[data['Weekday'] == day]  
    fig = plt.figure(figsize = (20,5))
    ax1 = fig.add_subplot(111)
    ax1.plot(y_week[day],color='steelblue',label=name[day])
    ax1.legend(loc='upper right')
    ax2 = ax1.twinx()
    ax2.plot(temp_week[day],color='coral',label=name[day]+'_temp')
    ax2.legend(loc='upper left')
    plt.title('Peaking Power and Min Temperature on Different Weekday on '+name[day])
    
    print(name[day]," ",y_week[day].corr(temp_week[day]))

  * 一週 7 天電力尖峰負載和最低溫度的相關係數和示意圖

# Model Selection

* 由上可知電力尖峰負載和最低溫度有高度正相關，又週末的數值會較週間低
* 初步想法：
    1. 使用 Linear Regression
    2. 一週中每一天分開做

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lm_trainX = [None] * 7
lm_trainY = [None] * 7
lm = [None] * 7
plt.figure(figsize = (20,20))
for day in range(7):
    lm_trainX[day] = np.reshape(temp_week[day].values,(len(temp_week[day].values), 1))
    lm_trainY[day] = np.reshape(y_week[day].values,(len(y_week[day].values), 1))
    lm[day] =  LinearRegression()
    lm[day].fit(lm_trainX[day], lm_trainY[day])
        
    plt.subplot(4, 3, day+1)
    plt.scatter(lm_trainX[day], lm_trainY[day])
    plt.plot( lm_trainX[day], lm[day].predict( lm_trainX[day]), color='black')
    
    plt.title('Peaking Power and Min Temperature on '+name[day])

* 從尖峰負載和溫度的散佈圖發現兩者的關係趨勢比起直線多了一點凹度，因此下面做了二次回歸

## Polynomial Features

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

lm_trainX = [None] * 7
lm_trainX2 = [None] * 7
lm_trainY = [None] * 7
lm2 = [None] * 7
poly = PolynomialFeatures(degree=2)
plt.figure(figsize = (20,20))
for day in range(7):
    lm_trainX[day] = np.reshape(temp_week[day].values,(len(temp_week[day].values), 1))
    lm_trainY[day] = np.reshape(y_week[day].values,(len(y_week[day].values), 1))
    poly.fit(lm_trainX[day])
    lm_trainX2[day] = poly.transform(lm_trainX[day])
    lm2[day] =  LinearRegression()
    lm2[day].fit(lm_trainX2[day], lm_trainY[day])
    
    y_predict2 = lm2[day].predict(lm_trainX2[day])
    plt.subplot(4, 3, day+1)
    plt.scatter(lm_trainX[day], lm_trainY[day])
    plt.plot( np.sort(temp_week[day].values), y_predict2[np.argsort(temp_week[day].values)], color='black')
    
    plt.title('Peaking Power and Min Temperature on '+name[day])   

### workday
* 將星期一到星期五合併做二次回歸

In [ ]:
y_workday =  y_data1718[(data['Weekday'] >= 0) & (data['Weekday'] <= 4)]
temp_workday = min_temper1718[(data['Weekday'] >= 0) & (data['Weekday'] <= 4)]  

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly = PolynomialFeatures(degree=2)

lm_workday_trainX = np.reshape(temp_workday.values,(len(temp_workday.values), 1))
lm_workday_trainY = np.reshape(y_workday.values,(len(y_workday.values), 1))
poly.fit(lm_workday_trainX)
lm_workday_trainX2 = poly.transform(lm_workday_trainX)
lm_workday2 =  LinearRegression()
lm_workday2.fit(lm_workday_trainX2, lm_workday_trainY)
y_predict = lm_workday2.predict(lm_workday_trainX2)
plt.figure()
plt.scatter(lm_workday_trainX, lm_workday_trainY)
plt.plot( np.sort(temp_workday.values), y_predict[np.argsort(temp_workday.values)], color='black')
 
plt.title('Polynomial Regression of Peaking Power and Min Temperature on Workday') 

### weekend

* 將星期六日合併做二次回歸

In [ ]:
y_weekend =  y_data1718[data['Weekday'] >=5]
temp_weekend = min_temper1718[data['Weekday'] >=5]  

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly = PolynomialFeatures(degree=2)

lm_weekend_trainX = np.reshape(temp_weekend.values,(len(temp_weekend.values), 1))
lm_weekend_trainY = np.reshape(y_weekend.values,(len(y_weekend.values), 1))
poly.fit(lm_weekend_trainX)
lm_weekend_trainX2 = poly.transform(lm_weekend_trainX)
lm_weekend2 =  LinearRegression()
lm_weekend2.fit(lm_weekend_trainX2, lm_weekend_trainY)
y_predict = lm_weekend2.predict(lm_weekend_trainX2)
plt.figure()
plt.scatter(lm_weekend_trainX, lm_weekend_trainY)
plt.plot( np.sort(temp_weekend.values), y_predict[np.argsort(temp_weekend.values)], color='black')
 
plt.title('Polynomial Regression of Peaking Power and Min Temperature on weekend') 

## Evaluation
* 以 2019/01 的資料比較不同種 model 方法的RMSE
    1. Linear Regression ( 線性回歸 ) / 一週 7 個 model
    2. Polynomial Features ( 二次回歸 ) / 一週 7 個 model
    3. Polynomial Features ( 二次回歸 ) / workday
    4. Polynomial Features ( 二次回歸 ) / weekend

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

y_data19 = y_data['2019']
min_temper19 = min_temper['2019']
y_week19 = [None] * 7
temp_week19 = [None] * 7

lm_testX = [None] * 7
lm_testY = [None] * 7
y_pred_test = [None] * 7
err_poly = [None] * 7
pred = []
ori = []

err_lr = [None] * 7 
for day in range(7):
    y_week19[day] = y_data19[data['Weekday'] == day]  
    temp_week19[day] = min_temper19[data['Weekday'] == day] 
    ori = np.hstack((ori,y_week19[day].values))
    
    lm_testX[day] = np.reshape(temp_week19[day].values,(len(temp_week19[day].values), 1))
    lm_testY[day] = np.reshape(y_week19[day].values,(len(y_week19[day].values), 1))
    y_pred_test[day] = lm[day].predict(lm_testX[day])
    pred = np.hstack((pred,np.hstack(y_pred_test[day])))
    err_lr[day] = rmse(y_pred_test[day].T,y_week19[day].values.T)
err_lr 

 * 線性回歸，一天一個 model - 一週七天

In [ ]:
err_lr_all = rmse(ori,pred)
err_lr_all

 * 線性回歸，一天一個 model - 整體

In [ ]:
lm_testX2 = [None] * 7
y_pred_test = [None] * 7
err_poly = [None] * 7
pred = []
ori = []
for day in range(7):
    ori = np.hstack((ori,y_week19[day].values))
    
    lm_testX[day] = np.reshape(temp_week19[day].values,(len(temp_week19[day].values), 1))
    lm_testY[day] = np.reshape(y_week19[day].values,(len(y_week19[day].values), 1))
    poly.fit(lm_testX[day])
    lm_testX2[day] = poly.transform(lm_testX[day])
    y_pred_test[day] = lm2[day].predict(lm_testX2[day])
    pred = np.hstack((pred,np.hstack(y_pred_test[day])))
    err_poly[day] = rmse(y_pred_test[day].T,y_week19[day].values.T)
err_poly

 * 二次回歸，一天一個 model - 一週七天

In [ ]:
err_poly_all = rmse(ori,pred)
err_poly_all

 * 二次回歸，一天一個 model - 整體

In [ ]:
ori = []
pred = []
for i in range(5):
    ori = np.hstack((ori, y_week19[i].values))
    pred = np.hstack((pred, np.hstack(y_pred_test[i])))
err_poly = rmse(ori,pred)
err_poly

 * 二次回歸，一天一個 model - 週一到五 - 整體

In [ ]:
ori = np.hstack((y_week19[5].values, y_week19[6].values))
pred = np.hstack((np.hstack(y_pred_test[5]), np.hstack(y_pred_test[6])))
err_poly = rmse(ori,pred)
err_poly

 * 二次回歸，週六日一天一 model - 整體

In [ ]:
y_workday19 = y_data19[(data['Weekday'] >= 0) & (data['Weekday'] <= 4)]  
temp_workday19 = min_temper19[(data['Weekday'] >= 0) & (data['Weekday'] <= 4)] 
    
lm_workday_testX = np.reshape(temp_workday19.values,(len(temp_workday19.values), 1))
lm_workday_testY = np.reshape(y_workday19.values,(len(y_workday19.values), 1))
poly.fit(lm_workday_testX)
lm_workday_testX2 = poly.transform(lm_workday_testX)
y_pred_test = lm_workday2.predict(lm_workday_testX2)
    
err_workday = rmse(y_pred_test.T,y_workday19.values.T)
err_workday

 * 二次回歸，週一到五一個 model

In [ ]:
y_weekend19 = y_data19[data['Weekday'] >= 5]  
temp_weekend19 = min_temper19[data['Weekday'] >= 5] 
    
lm_weekend_testX = np.reshape(temp_weekend19.values,(len(temp_weekend19.values), 1))
lm_weekend_testY = np.reshape(y_weekend19.values,(len(y_weekend19.values), 1))
poly.fit(lm_weekend_testX)
lm_weekend_testX2 = poly.transform(lm_weekend_testX)
y_pred_test = lm_weekend2.predict(lm_weekend_testX2)
    
err_weekend = rmse(y_pred_test.T,y_weekend19.values.T)
err_weekend

 * 二次回歸，週六日一個 model

* 由上面三筆 RMSE 發現
    1. 二次回歸整體而言效果較線性回歸好
    2. 將週一到週五用一個 model 預測，誤差較分成 5 個 model 的整體誤差差不多
    3. 將週末分成 2 個 model 預測，誤差明顯較用一個 model 的整體誤差小
* 因此決定採用 - 二次回歸 / 一天一個 mode

# Prediction

* 根據天氣預報中最高最低氣溫的平均預測該日電力尖峰負載
* [氣溫資料來源 -  4/1 的１週預報](https://www.cwb.gov.tw/V7/forecast/taiwan/Taipei_City.htm)

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y%m%d')
with open('201904_temp_gov.csv', 'r') as f:   
     weather1904 = pd.read_csv(f, parse_dates=['date'], date_parser=dateparse, index_col=0)
max_temper1904 = pd.Series(weather1904['T Max'],index=weather1904.index)
min_temper1904 = pd.Series(weather1904['T Min'],index=weather1904.index)
#temper1904 = (max_temper1904+min_temper1904)/2

weekdayr1904 = weather1904.index.weekday
weather1904.head(7)    

* 因為 2019/4/4 - 2019/4/7 是連假，根據 2017、2018 的清明連假數據顯示連假期間電力尖峰負載也會接近假日的趨勢，因此將他們的 weekday 改為星期六日日日

In [ ]:
weekdayr1904.values[2] = 5
weekdayr1904.values[3] = 6
weekdayr1904.values[4] = 6
weekdayr1904

### Test
 * 使用 台灣電力公司_未來一週電力供需預測 來測試

In [ ]:
y_1904 = [28400,28200,25700,24600,24300,24500,28300]
x_1904 = [None] * len(weather1904)
min_temper1904_2 = [None] * len(weather1904)
y_pred_1904 = np.zeros(len(weather1904))
ori = []
pred = []
for day in range(len(weather1904)):
    x_1904[day] = np.reshape(min_temper1904[day], (1, -1))
    ori = np.hstack((ori,y_1904[day]))
    poly.fit(x_1904[day])
    min_temper1904_2[day] = poly.transform(x_1904[day])
    y_pred_1904[day] = lm2[weekdayr1904[day]].predict(min_temper1904_2[day])
    pred = np.hstack((pred,y_pred_1904[day]))
    
y_pred_1904

In [ ]:
err_pred = [None] * 7 
for day in range(len(weather1904)):
    err_pred[day] = rmse(y_pred_1904[day],y_1904[day])
err_pred

In [ ]:
err_pred_all = rmse(ori, pred)
err_pred_all

* 和未來一週電力供需預測做 RMSE 結果 (分開 / 整體)

## Output result

In [ ]:
result = pd.DataFrame()
result['peak_load(MW)'] = (y_pred_1904).astype(np.int)
result.index = weather1904.index

result.head(7)   

In [ ]:
with open('submission.csv', 'w', newline='\n') as f:
    result.to_csv(f,date_format='%Y%m%d')